<a href="https://colab.research.google.com/github/sinheechan/Project_Yolov8/blob/main/Yolov8_seg_car_0227.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Source Code clone

In [1]:
# ultralytics 라이브러리 소스 코드 clone
!git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 23591, done.
remote: Counting objects: 100% (636/636), done.
remote: Compressing objects: 100% (443/443), done.
remote: Total 23591 (delta 377), reused 331 (delta 193), pack-reused 22955
Receiving objects: 100% (23591/23591), 14.21 MiB | 13.13 MiB/s, done.
Resolving deltas: 100% (16470/16470), done.


## 2. Route Settings

In [8]:
cd ultralytics/

/content/ultralytics/ultralytics


In [9]:
pwd

'/content/ultralytics/ultralytics'

## 3. Install

In [ ]:
!pip install ultralytics

## 4. Yolo Predict

In [ ]:
!yolo predict

WARNING ⚠️ 'model' is missing. Using default 'model=yolov8n.pt'.
100% 6.23M/6.23M [00:00<00:00, 120MB/s]
WARNING ⚠️ 'source' is missing. Using default 'source=/usr/local/lib/python3.10/dist-packages/ultralytics/assets'.
Ultralytics YOLOv8.1.19 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

image 1/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 96.8ms
image 2/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/zidane.jpg: 384x640 2 persons, 1 tie, 87.4ms
Speed: 7.4ms preprocess, 92.1ms inference, 339.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


## 5. Dataset Download_Roboflow

In [ ]:
!curl -L "https://universe.roboflow.com/ds/Hxie9vewYe?key=hbcFa5gtFP"> roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   903  100   903    0     0   3537      0 --:--:-- --:--:-- --:--:--  3541
100  877M  100  877M    0     0  51.7M      0  0:00:16  0:00:16 --:--:-- 82.5M


In [ ]:
!unzip roboflow.zip

## 6. Train / Val Dataset

- train set / val set 이미지와 레이블 파일을 각각 변수에 저장한다.

In [ ]:
from glob import glob

train_img = glob('/content/ultralytics/train/images/*.jpg')
train_txt = glob('/content/ultralytics/train/labels/*.txt')

val_img = glob('/content/ultralytics/valid/images/*.jpg')
val_txt = glob('/content/ultralytics/valid/labels/*.txt')

print(len(train_img), len(train_txt))
print(len(val_img), len(val_txt))

6603 6603
1887 1887


- train_text + train_image : text_image 파일의 경로가 train_text 파일 각 줄에 기록
- val_text + val_image : val_image 파일의 경로가 val_text 파일 각 줄에 기록

In [ ]:
# 파일 쓰기 # train.txt, val.txt 파일 생성
with open('/content/ultralytics/train.txt', 'w') as f:
  f.write('\n'.join(train_img) + '\n')

with open('/content/ultralytics/val.txt', 'w') as f:
  f.write('\n'.join(val_img) + '\n')

## 7. yaml 수정

- data.yaml 파일을 읽고 파이썬 데이터 구조로 로드한 후, 데이터 구조를 수정하여 새로운 경로를 할당

In [ ]:
# yaml 수정
# 내 train, val 자료셋이 어디있는지 yaml 파일 안에 기록해줘야함 / 수동으로 기입해도 상관은 없다.
import yaml

with open('/content/ultralytics/data.yaml') as f:
  data = yaml.full_load(f)

print(data)
data['train'] = '/content/ultralytics/train.txt'
data['val'] = '/content/ultralytics/val.txt'

with open('/content/ultralytics/data.yaml', 'w') as f:
  yaml.dump(data, f)

{'train': '../train/images', 'val': '../valid/images', 'test': '../test/images', 'nc': 14, 'names': ['Bonnet', 'Frontbumper', 'Frontdoor', 'Frontfender', 'Headlights', 'Rearbumper', 'Reardoor', 'Rearfender', 'Rearlamp', 'Rockerpanel', 'Sidemirror', 'Trunklid', 'Wheel', 'Windshield'], 'roboflow': {'workspace': 'alpaco5-f3woi', 'project': 'part-autolabeld', 'version': 5, 'license': 'Public Domain', 'url': 'https://universe.roboflow.com/alpaco5-f3woi/part-autolabeld/dataset/5'}}


## 8. Segmentation train

- 모델을 사용하여 이미지를 세그멘테이션(segmentation)하는 작업을 수행한다.

- 세그맨테이션 | 학습 옵션 | YOLO 모델 지정 | 데이터 지정 | 에포크 = 10

In [ ]:
!yolo segment train model=yolov8s-seg.pt data=/content/ultralytics/data.yaml epochs=10

100% 22.8M/22.8M [00:00<00:00, 239MB/s]
Ultralytics YOLOv8.1.19 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=/content/ultralytics/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cr